# Guia de Tooling com LangChain

Este notebook expande a aula para focar em ferramentas (tooling), agentes e segurança, mantendo o estilo de tutoriais e dicas.

Assuntos:
- Definição de ferramentas customizadas para descrever parâmetros.
- Uso de ferramentas prontas da LangChain.
- Compreendendo o fluxo de tool_calling/function_calling.
- Técnicas de segurança para a execução de ferramentas.


In [ ]:
%pip install -q -r ../requirements.txt

In [1]:
import os
from typing import List, Optional

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_google_genai import ChatGoogleGenerativeAI

# Inicialização do LLM
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
model_name = os.getenv("MODEL_NAME", "gemini-2.0-flash")
assert api_key, "GOOGLE_API_KEY ausente. Defina no .env."
llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=api_key, streaming=False)
print("LLM pronto.")


LLM pronto.


## 1) Ferramentas customizadas (StructuredTool + Pydantic)

Objetivo: definir ferramentas com parâmetros tipados e descrições claras, facilitando a chamada pelo agente.


In [2]:
from langchain.tools import StructuredTool

class CalculaMediaArgs(BaseModel):
    numeros: List[float] = Field(..., description="Lista de números para média")

def safety_guard(text: str) -> None:
    bloqueados = ["rm -rf", "drop table", "shutdown", "format c:\\"]
    low = str(text).lower()
    if any(b in low for b in bloqueados):
        raise ValueError("Conteúdo potencialmente perigoso bloqueado.")

def calcula_media(numeros: List[float]) -> str:
    # guardas de segurança em entradas não confiáveis
    safety_guard(numeros)
    if not numeros:
        return "Lista vazia; informe ao menos um número."
    media = sum(numeros) / len(numeros)
    return f"Média: {media:.4f}"

calcula_media_tool = StructuredTool.from_function(
    name="calcula_media",
    description="Calcula a média aritmética com checagem de segurança.",
    func=calcula_media,
    args_schema=CalculaMediaArgs,
)

print(calcula_media_tool.invoke({"numeros": [10, 5, 7]}))


Média: 7.3333


## 2) Ferramentas prontas (langchain-community)

Exemplo com DuckDuckGo: útil para demonstrações rápidas de integração com web search.

Dica: em ambientes sem internet, esta ferramenta pode falhar. Foque em ferramentas locais/customizadas.


In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun(name="web_search")
# Exemplo simples de uso direto
try:
    print(search_tool.run("Novidades LangChain 2024"))
except Exception as e:
    print("Falha na busca (rede indisponível?):", e)


c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


No good DuckDuckGo Search Result was found


## 3) Fluxo de tool_calling/function_calling com agente

Montaremos um agente que decide quando e como chamar ferramentas com base na instrução.

Dica: descreva bem "name" e "description" das ferramentas; isso ajuda o LLM a escolher corretamente.


In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

tools = [calcula_media_tool]
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de suporte. Use ferramentas quando necessário e siga as descrições e nomes. Responda de forma sucinta."),
    ("human", "{in}"),
    MessagesPlaceholder("agent_scratchpad"),
])
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Solicitação composta: cálculo + pesquisa
query = "Calcule a média de 10, 5 e 7 e depois busque novidades sobre LangChain."
res = executor.invoke({"in": query})
print(res.get("output") or res)




> Entering new AgentExecutor chain...
{'name': 'calcula_media', 'description': 'Calcula a média aritmética com checagem de segurança.', 'parameters': {'properties': {'numeros': {'description': 'Lista de números para média', 'items': {'type': 'number'}, 'type': 'array'}}, 'required': ['numeros'], 'type': 'object'}}

Invoking: `calcula_media` with `{'numeros': [10.0, 5.0, 7.0]}`


Média: 7.3333{'name': 'calcula_media', 'description': 'Calcula a média aritmética com checagem de segurança.', 'parameters': {'properties': {'numeros': {'description': 'Lista de números para média', 'items': {'type': 'number'}, 'type': 'array'}}, 'required': ['numeros'], 'type': 'object'}}
A média é 7.3333. Não posso buscar notícias.

> Finished chain.
A média é 7.3333. Não posso buscar notícias.


## 4) Técnicas de segurança

Ideias de mitigação:
- Guardas de segurança (allowlist/blocklist) nas entradas das ferramentas.
- Sanitização de parâmetros (limites de tamanho, normalização).
- Logs de auditoria e "dry-run" para ferramentas sensíveis.
- Restrição do conjunto de ferramentas acessíveis conforme o contexto.


In [5]:
class CriaTicketArgs(BaseModel):
    titulo: str = Field(..., description="Título do ticket")
    prioridade: Optional[str] = Field(default="media", description="Prioridade: baixa|media|alta")

def cria_ticket(titulo: str, prioridade: str = "media") -> str:
    safety_guard(titulo)
    pr = prioridade.lower()
    if pr not in {"baixa", "media", "alta"}:
        pr = "media"
    # Sanitização de tamanho
    if len(titulo) > 140:
        titulo = titulo[:140]
    return f"Ticket criado: TCK-{abs(hash(titulo)) % 10000} ({pr}) — '{titulo}'"

cria_ticket_tool = StructuredTool.from_function(
    name="cria_ticket",
    description="Cria um ticket fictício com validação de prioridade e sanitização do título.",
    func=cria_ticket,
    args_schema=CriaTicketArgs,
)

# Demonstração de segurança
print(cria_ticket_tool.invoke({"titulo": "Erro 500 ao logar no app", "prioridade": "alta"}))
try:
    print(cria_ticket_tool.invoke({"titulo": "rm -rf /"}))
except Exception as e:
    print("Bloqueado por segurança:", e)


Ticket criado: TCK-5664 (alta) — 'Erro 500 ao logar no app'
Bloqueado por segurança: Conteúdo potencialmente perigoso bloqueado.


## Dicas finais

- Preferir ferramentas determinísticas e idempotentes.
- Documente claramente parâmetros e efeitos colaterais.
- Em produção, use observabilidade (logs, métricas) para ferramentas sensíveis.
- Teste ferramentas isoladamente antes de integrá-las ao agente.


## 6) Mais dicas

- Exija parâmetros mínimos e valide formatos com Pydantic.
- Evite efeitos colaterais irreversíveis; inclua modo dry-run para produção.
- Padronize o nome e descrição das ferramentas; isso ajuda o LLM a escolher.
- Use testes unitários para cada ferramenta e testes de integração para o agente.


## 7) Ferramentas com APIs abertas

Exemplo de ferramenta que consome uma API pública sem autenticação (JSONPlaceholder).

In [10]:
import requests
from typing import Optional
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class BuscaPostsArgs(BaseModel):
    user_id: Optional[int] = Field(default=None, description="Filtrar por usuário (opcional)")
    limit: int = Field(default=5, ge=1, le=20, description="Número máximo de posts a retornar")

def busca_posts(user_id: Optional[int] = None, limit: int = 5) -> str:
    params = {}
    if user_id is not None:
        params['userId'] = int(user_id)
    r = requests.get('https://jsonplaceholder.typicode.com/posts', params=params, timeout=10)
    r.raise_for_status()
    data = r.json()[:limit]
    linhas = [f"[{p['id']}] {p['title']}" for p in data]
    return "".join(linhas) if linhas else "Nenhum post encontrado"

busca_posts_tool = StructuredTool.from_function(
    name="busca_posts",
    description="Busca posts em API pública (JSONPlaceholder).",
    func=busca_posts,
    args_schema=BuscaPostsArgs,
)

print(busca_posts_tool.invoke({"user_id": 1, "limit": 3}))


[1] sunt aut facere repellat provident occaecati excepturi optio reprehenderit[2] qui est esse[3] ea molestias quasi exercitationem repellat qui ipsa sit aut


## 8) Integrações populares (ex.: clima)

Exemplo com OpenWeatherMap (requer `OPENWEATHER_API_KEY` no `.env`).

In [11]:
import os, requests
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class OpenWeatherArgs(BaseModel):
    cidade: str = Field(..., description="Cidade para consulta (ex.: São Paulo)")
    unidades: str = Field(default="metric", description="Unidades: metric|imperial")

def consulta_clima(cidade: str, unidades: str = "metric") -> str:
    key = os.getenv('OPENWEATHER_API_KEY')
    if not key:
        return "OPENWEATHER_API_KEY ausente no ambiente/.env."
    url = 'https://api.openweathermap.org/data/2.5/weather'
    params = {'q': cidade, 'appid': key, 'units': unidades, 'lang': 'pt_br'}
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return f"Falha na API ({r.status_code}): {r.text[:120]}"
    j = r.json()
    desc = j.get('weather', [{}])[0].get('description', 'sem descrição')
    temp = j.get('main', {}).get('temp', '?')
    feels = j.get('main', {}).get('feels_like', '?')
    return f"Tempo em {cidade}: {desc}, temp {temp}°C, sensação {feels}°C"

weather_tool = StructuredTool.from_function(
    name="consulta_clima",
    description="Consulta clima atual via OpenWeatherMap.",
    func=consulta_clima,
    args_schema=OpenWeatherArgs,
)

print(weather_tool.invoke({"cidade": "São Paulo"}))


OPENWEATHER_API_KEY ausente no ambiente/.env.


## 9) Ferramentas da comunidade (Wikipedia, Arxiv)

Instalação leve e exemplos de uso.

In [ ]:
%pip install -q wikipedia arxiv
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun

wiki_tool = WikipediaQueryRun()
arxiv_tool = ArxivQueryRun()

try:
    print(wiki_tool.run("LangChain"))
except Exception as e:
    print("Falha no Wikipedia:", e)

try:
    print(arxiv_tool.run("prompt engineering"))
except Exception as e:
    print("Falha no Arxiv:", e)


## 10) Composição ampliada no agente

Demonstração de agente com ferramentas customizadas e da comunidade.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools_ext = [
    calcula_media_tool,
    cria_ticket_tool,
    busca_posts_tool,
    weather_tool,
]
# Adiciona da comunidade se disponíveis
try:
    tools_ext += [wiki_tool, arxiv_tool]
except NameError:
    pass

prompt_ext = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de suporte. Use ferramentas com parcimônia e explique brevemente os resultados."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
agent_ext = create_tool_calling_agent(llm, tools_ext, prompt_ext)
executor_ext = AgentExecutor(agent=agent_ext, tools=tools_ext, verbose=True)

pedido = (
    "Busque 2 posts do usuário 1, depois crie um ticket prioridade media título: 'Instabilidade', e por fim consulte clima em São Paulo."
)
res_ext = executor_ext.invoke({"input": pedido})
print(res_ext.get("output") or res_ext)
